<a href="https://colab.research.google.com/github/SucyLi/ARC/blob/master/tplot_resnet50_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
import numpy as np
from keras import layers, models
from keras import optimizers
from keras.applications import ResNet50
from keras.callbacks import TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Softmax, Flatten, Dense, BatchNormalization 
from keras.metrics import categorical_accuracy
from keras import backend as K
from keras import regularizers
import tensorflow as tf
from keras.models import Sequential
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn import metrics
from sklearn.metrics import roc_auc_score

Using TensorFlow backend.


In [3]:
# train_dir = "/gdrive/My Drive/VisualScience/tplot/thickness_plot/train/"
# val_dir = "/gdrive/My Drive/VisualScience/tplot/thickness_plot/val/"
# test_dir = "/gdrive/My Drive/VisualScience/tplot/thickness_plot/test"

root_dir = "/gdrive/My Drive/VisualScience/Heidelberg/2020/2575"
train_dir = root_dir+"/tplot/train"
val_dir = root_dir+"/tplot/valid"
test_dir = root_dir+"/tplot/test"
test_maps_dir = root_dir+"/tplot/maps"

img_width = 768
img_height = 330
channels = 3
batch_size = 1
epochs = 30

train_datagen = ImageDataGenerator(rescale=1./255)             
valid_datagen = ImageDataGenerator(rescale=1./255)    
test_datagen = ImageDataGenerator(rescale=1./255) 
test_maps_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(img_height, img_width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True)   

valid_generator = valid_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True) #weight toward one class or another

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)

test_maps_generator = test_maps_datagen.flow_from_directory(
    test_maps_dir,
    target_size=(img_height, img_width),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)

nb_train_samples = train_generator.__len__()
nb_valid_samples = valid_generator.__len__()
nb_test_samples = test_generator.__len__()
nb_test_maps_samples = test_maps_generator.__len__()

label_mapT = train_generator.class_indices
print(label_mapT)

Found 398 images belonging to 2 classes.
Found 136 images belonging to 2 classes.
Found 139 images belonging to 2 classes.
Found 127 images belonging to 2 classes.
{'G': 0, 'NG': 1}


In [4]:
#pretrained ResNet50 on imagenet, starting to replicate Hassan's work
conv_base = ResNet50(weights='imagenet',
                  include_top=False,
                  input_shape=(img_height, img_width, channels))
conv_base.summary()

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 3s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 330, 768, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 336, 774, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 165, 384, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 165, 384, 64) 256         conv1[0][0]                      
__________________________

In [0]:
def extract_features(generator, sample_count):
    features = np.zeros(shape=(sample_count, 11, 24, 2048))
    labels = np.zeros(shape=(sample_count))
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break   
    return features, labels

In [0]:
def predict(generator, nb_samples, classifier, trainResult, tag):

  features, labels = extract_features(generator, nb_samples)

  prediction = clf.predict(np.reshape(features, (nb_samples, 11*24*2048)))
  print(tag + " accuracy:", sum([prediction[i] == labels[i] for i in range(len(labels))])/len(labels))

  score = trainResult.predict_proba(np.reshape(features, (nb_samples, 11*24*2048)))

  FP_list = []
  FP_ind = []
  FN_list = []
  FN_ind = []
  filenames = generator.filenames

  #FP
  for i in range(len(labels)):
    if labels[i] == 1 and prediction[i] == 0:
      FP_list.append(filenames[i])
      FP_ind.append(i)

  #FN
  for i in range(len(labels)):
    if labels[i] == 0 and prediction[i] == 1:
      FN_list.append(filenames[i])
      FN_ind.append(i)

  FP = len(FP_list) 
  FN = len(FN_list)

  print('FP num:', FP)
  print('FP list:', FP_list)
  print('FN num:', FN)
  print('FN list:', FN_list)

In [7]:
num_run = 1

for j in range(num_run):
    
  train_features, train_labels = extract_features(train_generator, nb_train_samples)

  clf = RandomForestClassifier(n_estimators=100, max_depth=12, random_state=0, bootstrap=True)
  trainResult = clf.fit(np.reshape(train_features, (nb_train_samples, 11*24*2048)), train_labels)

  predict(valid_generator, nb_valid_samples, clf, trainResult, "valid")
  predict(test_generator, nb_test_samples, clf, trainResult, "test")
  predict(test_maps_generator, nb_test_maps_samples, clf, trainResult, "maps")

valid accuracy: 0.9044117647058824
FP num: 4
FP list: ['G/P0385_OS_2-12-2018_Extra.png', 'NG/3-064_OS_ETH.png', 'NG/4-006_OD_CAU.png', 'NG/4-021_OD_CAU.png']
FN num: 9
FN list: ['G/P0359_OS_9-20-2016_Extra.png', 'G/P0378_OS_11-11-2016_Extra.png', 'G/P0403_OD_GL.png', 'G/P0436_OD_2-17-2017_Extra.png', 'G/P0446_OS_GL.png', 'G/P0461_OS_GL.png', 'G/P445_OD_12-11-2018_Extra.png', 'NG/4-007_OD_CAU.png', 'NG/4-016_OS_CAU.png']
test accuracy: 0.9712230215827338
FP num: 4
FP list: ['NG/P0420_OD_2-6-2017_Extra.png', 'NG/P0477_OS_4-13-2017_Extra.png', 'NG/P0544_OS_10-5-2017_Extra.png', 'NG/P406_OS_1-20-2017_Extra.png']
FN num: 0
FN list: []
maps accuracy: 0.7086614173228346
FP num: 9
FP list: ['NG/COL081_OD_5-17-2016_Extra.png', 'NG/COL130_OS_9-20-2016_Extra.png', 'NG/COL143_OS_1-11-2017_Extra.png', 'NG/COL151_OS_1-18-2017_Extra.png', 'NG/COL165_OD_2-13-2017_Extra.png', 'NG/COL167_OS_2-27-2017_Extra.png', 'NG/COL187_OS_7-10-2017_Extra.png', 'NG/COL199_OS_3-27-2017_Extra.png', 'NG/COL201_OD_4-21-2